#### select control

In [7]:
import random

sig_dir = "/Charles/project/ASm6A/ASm6A/homer_anno/split_region/"
# unsig_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/unsig/rename/homer_anno/split_region/"
unsig_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/rename/unsig/homer_anno/split_region/"
result_dir = "/Charles/project/ASm6A/ASm6A/control/"

def read_file(in_file):
    snp_list = pd.read_table(in_file, header=None).iloc[:,0].tolist()
    return list(set(snp_list))

def generate_SNP_list(asm6a_dir):
    os.chdir(asm6a_dir)
    num_dict, snp_dict = {}, {}
    file_list = glob.glob("*.txt")
    for i_file in file_list:
        region = i_file.split("_SNPid.txt")[0]
        num_dict[region] = len(read_file(i_file))
        snp_dict[region] = read_file(i_file)
    return num_dict, snp_dict
    
def write_to_file(snp_list, result_file):
    with open(result_file, 'w') as fw:
        for x in snp_list:
            chro, end = x.split(":")[0], int(x.split(":")[1])
            fw.write("%s\t%s\t%s\n" % (chro, str(end-1), str(end)))
    
def random_select_sets(asm6a_dir, sample):
    sig_num_dict, sig_snp_dict = generate_SNP_list(asm6a_dir)
    unsig_num_dict, unsig_snp_dict = generate_SNP_list(os.path.join(unsig_dir, sample))
    result_list = []
    for region, num in sig_num_dict.items():
        sampled_snps = []
        if region in unsig_snp_dict:  ## unsig SNP didn't exist in this region
            snp_list = unsig_snp_dict[region]
            if len(snp_list) < num:   ## the number of unsig SNPd less than sig SNPs
                sampled_snps = snp_list
                print(sample, region, num)
            else:
                sampled_snps = random.sample(snp_list, num)
        result_list += sampled_snps
    result_file = os.path.join(result_dir, "%s.bed"%sample)
    write_to_file(result_list, result_file)

dir_list = [x for x in glob.glob("%s/*" % sig_dir) if os.path.isdir(x)]
for asm6a_dir in dir_list:
    sample = asm6a_dir.split("/")[-1]
    # if sample == "brain_IP_1":
    random_select_sets(asm6a_dir, sample)